In [7]:
#for image processing
import cv2 

#for standard image re-sizing(lowers processing time)
import imutils


for idx in range(1,4):
    # Read
    image = cv2.imread('Cars/' + str(idx) + '.jpeg')
    # Resize
    image = imutils.resize(image, width=500) #gets it into feasible aspect ratio

    #original image
    cv2.imshow("Original Image", image)
    cv2.waitKey(0) #waits till a key is pressed

    # RGB to Gray scale conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("1 - Grayscale Conversion", gray)
    #cv2.waitKey(0)

    # Remove Noise, Sharpness (Averaging)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    #cv2.imshow("2 - Bilateral Filter", gray)
    #cv2.waitKey(0)

    # Find Edges of the grayscale image - Canny Edge Detection 
    edged = cv2.Canny(gray, 170, 200)
    #cv2.imshow("3 - Canny Edges", edged)
    #cv2.waitKey(0)

    # Find number contours based on Edges
    cnts, new  = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create copy of original image to draw all contours
    #we create a copy so as to not destroy the original image
    img1 = image.copy()
    cv2.drawContours(img1, cnts, -1, (0,255,0), 3)
    #cv2.imshow("4- All Contours", img1)
    #cv2.waitKey(0)

    #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
    #max to min sorting
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
    NumberPlateCnt = None 

    # Top 30 Contours
    img2 = image.copy()
    cv2.drawContours(img2, cnts, -1, (0,255,0), 3)
    #cv2.imshow("5- Top 30 Contours", img2)
    #cv2.waitKey(0)

    # loop over each of thr top 30 contours 
    #find the best possible approximate contour of number plate
    
    for c in cnts:
            peri = cv2.arcLength(c, True) #find perimeter
            approx = cv2.approxPolyDP(c, 0.02 * peri, True) #no. of edges for each contour
            if len(approx) == 4:  # Select the contour with 4 corners
                NumberPlateCnt = approx #This is our approx Number Plate Contour

                #crop according to coordinates - our ROI
                x, y, w, h = cv2.boundingRect(c) #This will find out co-ord for plate
                new_img = gray[y:y + h, x:x + w] #crop the grayscale image
                #cv2.imwrite('BoundedBox/' + str(idx) + '.jpg', new_img) #Store new image
                #idx+=1

                break

    # Drawing the selected contour on the original image
    print(NumberPlateCnt) #total number of plates in the picture and their coordinates
    cv2.drawContours(image, [NumberPlateCnt], -1, (0,255,0), 3)
    #cv2.imshow("Final Image With Number Plate Detected", image)
    #cv2.waitKey(0)
    
    cv2.imwrite('BoundedBox/' + str(idx) + '.jpg', image) #Store new image
    cv2.imshow("Cropped Image ", new_img)
    cv2.waitKey(0)
    #print(idx)
    #idx = idx + 1
    

[[[364 223]]

 [[188 223]]

 [[185 259]]

 [[362 258]]]
None


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2606: error: (-215:Assertion failed) reader.ptr != NULL in function 'cvDrawContours'
